In [10]:
import re,codecs,unidecode

In [2]:
fJoueurs="NomsJoueurs.txt"
with codecs.open(fJoueurs,"r",encoding="utf8") as inFile:
    lJoueurs="".join(inFile.readlines())

In [3]:
tEquipes=re.split(u"\n(?=.*:)",lJoueurs)

In [4]:
equipes={}
for tEquipe in tEquipes[1:]:
    print tEquipe
    lEquipe=tEquipe.split(u"\n")
    nomEquipe=lEquipe[0].split(":")[0].strip()
    equipes[nomEquipe]={}
    equipe=equipes[nomEquipe]
    for ligne in lEquipe[1:]:
        m=re.match(ur"(.*)\t(.*)",ligne)
        if m:
            print "poste",m.group(1)
            print "joueurs",m.group(2).split(u",")
            poste=m.group(1).strip()
            joueurs=[j.strip() for j in m.group(2).split(u",") if j.strip()!=""]
            if joueurs:
                equipe[poste]=joueurs
    

Angleterre : 

Entraîneur	Brian Ashton
Avants	
	Pilier	Perry Freshwater, Andrew Sheridan, Matt Stevens, Phil Vickery
	Talonneur	George Chuter, Lee Mears, Mark Regan
	Deuxième ligne	Steve Borthwick, Ben Kay, Simon Shaw
	Troisième ligne aile	Lewis Moody, Tom Rees, Joe Worsley
	Troisième ligne centre	Martin Corry, Lawrence Dallaglio, Nick Easter
Arrières	
	Demi de mêlée	Andy Gomarsall, Shaun Perry, Peter Richards
	Demi d'ouverture	Jonny Wilkinson, Olly Barkley
	Centre	Mike Catt, Andy Farrell, Dan Hipkiss, Jamie Noon, Mathew Tait, Toby Flood
	Ailier	Jason Robinson, Paul Sackey
	Arrière	Mark Cueto, Josh Lewsey



poste Entraîneur
joueurs [u'Brian Ashton\r']
poste Avants
joueurs [u'\r']
poste 	Pilier
joueurs [u'Perry Freshwater', u' Andrew Sheridan', u' Matt Stevens', u' Phil Vickery\r']
poste 	Talonneur
joueurs [u'George Chuter', u' Lee Mears', u' Mark Regan\r']
poste 	Deuxième ligne
joueurs [u'Steve Borthwick', u' Ben Kay', u' Simon Shaw\r']
poste 	Troisième ligne aile
joueurs [u'Lewis Moo

In [5]:
def getJoueurs(pays):
    result=[]
    for k,v in equipes[pays].iteritems():
        result.extend(v)
        result=[j.replace(u"(cap.)","").strip() for j in result]
    return result

In [6]:
joueursFr=getJoueurs("France")
joueursAr=getJoueurs("Argentine")
autres=[u"Thierry Lacroix",u"Fabrice Landreau",u"Thierry Gilardi"]

In [7]:
def ajoutNP(lSet,np):
    decNP=unicode(unidecode.unidecode(np))
    lSet.add(np)
    if np!=decNP:
        lSet.add(decNP)

In [8]:
def faireRegexJoueurs(joueurs):
    listeTemp=set()
    for j in joueurs:
        listeTemp.add(j)
        listeTemp.add(unidecode.unidecode(j))
    nomsComplets=u"|".join(sorted(listeTemp))
    prenoms=set()
    noms=set()
    for j in joueurs:
        prenom,nom=j.split(" ",1)
        ajoutNP(prenoms,prenom)
        ajoutNP(noms,nom)
        if " " in nom:
            nomParts=nom.split(" ")
            for n in nomParts:
                if len(n)>2:
                    ajoutNP(noms,n)
#    prenomsSeuls="|".join(sorted(prenoms|noms,key=lambda x:len(x),reverse=True))
#    nomsSeuls="|".join(sorted(noms,key=lambda x:len(x),reverse=True))
    nomsSepares=u"|".join(sorted(prenoms|noms,key=lambda x:len(x),reverse=True))
    return nomsComplets,nomsSepares

In [11]:
nomsCompletsFr,nomsSeparesFr=faireRegexJoueurs(joueursFr)
nomsCompletsFr=nomsCompletsFr.replace(u"Frédéric Michalak",u"Frédéric Michalak|Fred Michalak")
coordJoueursFr=u"%s|%s"%(nomsCompletsFr,nomsSeparesFr)
nomsCompletsAr,nomsSeparesAr=faireRegexJoueurs(joueursAr)
coordJoueursAr=u"%s|%s"%(nomsCompletsAr,nomsSeparesAr)
nomsCompletsOt,nomsSeparesOt=faireRegexJoueurs(autres)
#regexJoueurs=u"(%s|%s|%s|%s)"%(nomsCompletsFr,nomsCompletsAr,nomsSeparesFr,nomsSeparesAr)
regexJoueurs=u"(%s|%s|%s|%s|%s|%s)"%(nomsCompletsFr,nomsCompletsAr,nomsCompletsOt,nomsSeparesFr,nomsSeparesAr,nomsSeparesOt)

In [12]:
from lxml import etree
from pydub import AudioSegment

In [16]:
repRugby="/Volumes/BroadDisk/Rugby/"
repTRS="/Users/gilles/ownCloud/Recherche/Rugby/"
repWAV="/Users/gilles/pCloud Drive/FOD/Copy/Recherche/Rugby/"
#fichierTranscription=open(repRugby+"France_Argentine_ANNOT-REF_essai.trs","r")
fichierTranscription=open(repTRS+"France_Argentine_Intro_FR.trs","r")
xmlTranscription=fichierTranscription.read()
sound = AudioSegment.from_wav(repWAV+"France_Argentine_match_Temp.wav")

In [17]:
root=etree.XML(xmlTranscription)

In [18]:
Topics=root[1]
Episodes=root[2]
sections=[]
for section in Episodes.findall('Section'):
    sectionType=section.get('type')
    sectionTopic=section.get('topic')
    sectionStart=section.get('startTime')
    sectionEnd=section.get('endTime')
    turns=[]
    for turn in section.findall('Turn'):
        speaker=turn.get('speaker')
        if speaker:
            speakers=speaker.split(" ")
        else:
            speakers=[""]
        startTime=turn.get('startTime')
        endTime=turn.get('endTime')
        syncs=[]
        whos=0
        for element in turn.iter():
            if element.tag=="Sync":
                syncs.append((element.get('time'),speakers[0]))
            elif element.tag=="Who":
                whos+=1
                if whos>1:
                    syncs.append((syncs[-1][0],speakers[-1]))
            elif element.tag!="Turn":
            	contents=[element.tag]
            	for attrib in element.attrib:
            		contents.append(attrib)
            		contents.append(element.attrib[attrib])
            	content=" ".join(contents)
                syncs.append((syncs[-1][0],content))
        texts=[]
        for text in turn.itertext():
            if text!="\n":
                texts.append(text.strip("\n"))
        elements=[]
        for i in range(len(texts)):
            elements.append((syncs[i][0],syncs[i][1],texts[i]))
        turns.append((startTime,endTime,speaker,elements))
    sections.append((sectionType,sectionTopic,sectionStart,sectionEnd,turns))

In [21]:
printSelection="Sync"
nbToursNAME=0
nbToursCourtsNAME=0
nbToursExtraCourtsNAME=0
numSection=0
numTurn=0
numSync=0
for stype,stopic,sstart,send,turns in sections:
    numSection+=1
#    print "Section",numSection,stype,stopic,sstart,send
    if printSelection=="Section":
        print sstart,";",send,";",stype,";",stopic
    numTurnIn=0
    for tstart,tend,speaker,text in turns:
        numTurn+=1
        numTurnIn+=1
#        print "\tTurn",numTurn,tstart, tend, speaker
        if printSelection=="Turn":
            print tstart,";",tend,";",speaker
        numSyncIn=0
        for sync in text:
            numSync+=1
            numSyncIn+=1
#            print "\t\tSync",numSync, sync[0], sync[1], sync[2]
            m=re.search(regexJoueurs,sync[2])
            if printSelection=="Sync" and m:
                nbToursNAME+=1
                nbMots=len(sync[2].split(" "))
                if nbMots<5 or 1:
#                    nbToursCourtsNAME+=1
                    if nbMots<4 or 1:
#                        nbToursExtraCourtsNAME+=1
                        tour=sync[2]
                        print sync[0],";", sync[1],";", sync[2].encode("utf8")
                        print m.group(1)
                        fStart=float(tstart)
                        fEnd=float(tend)
                        sStart=int(fStart*1000)
                        sEnd=int(fEnd*1000)
                        print sStart,sEnd
                        extrait=sound[sStart:sEnd]
                        nomSon=u"%08.3f-%08.3f-%s.wav"%(fStart,fEnd,"".join([m.capitalize() for m in tour.split(" ")]))
                        print nomSon
                        extrait.export(repTRS+nomSon, format="wav")


37.086 ; spk1 ; est donné par David Skrela.
David Skrela
37086 39447
0037.086-0039.447-EstDonnéParDavidSkrela..wav
43.213 ; spk1 ; avec un départ de Roncero
Roncero
43213 44769
0043.213-0044.769-AvecUnDépartDeRoncero.wav
45.468 ; spk1 ; Pichot a appelé le ballon 
Pichot
45468 46684
0045.468-0046.684-PichotAAppeléLeBallon.wav
47.456 ; spk1 ; pour Hernandez
Hernandez
47456 48706
0047.456-0048.706-PourHernandez.wav
72.362 ; spk1 ; pour le capitaine de l'équipe de France Raphaël Ibañez
Raphaël Ibañez
72362 74730
0072.362-0074.730-PourLeCapitaineDeL'équipeDeFranceRaphaëlIbañez.wav
82.103 ; spk1 ; et ça profite à Pichot
Pichot
82103 83293
0082.103-0083.293-EtÇaProfiteÀPichot.wav
85.631 ; spk1 ; Cédric Heymans en couverture il va devoir
Cédric Heymans
85631 89034
0085.631-0089.034-CédricHeymansEnCouvertureIlVaDevoir.wav
89.358 ; spk1 ; oui avec la cuillère de Pichot 
Pichot
89358 91358
0089.358-0091.358-OuiAvecLaCuillèreDePichot.wav
91.83 ; spk1 ; Heymans est au sol 
Heymans
91830 92809
0091.

749.422 ; spk1 ; avec Pieter de Villiers
Pieter de Villiers
749422 750769
0749.422-0750.769-AvecPieterDeVilliers.wav
751.519 ; spk1 ; avec Heymans maintenant c'est la grande relance
Heymans
751519 754206
0751.519-0754.206-AvecHeymansMaintenantC'estLaGrandeRelance.wav
754.578 ; spk1 ; française jusqu'à Rougerie
Rougerie
754578 756209
0754.578-0756.209-FrançaiseJusqu'àRougerie.wav
758.618 ; spk1 ; Mignoni du rythme du rythme
Mignoni
758618 760505
0758.618-0760.505-MignoniDuRythmeDuRythme.wav
761.66 ; Event desc pi type pronounce extent instantaneous ;  Rémy Martin
Rémy Martin
761660 763272
0761.660-0763.272-RémyMartin.wav
785.367 ; spk2 ; avec Betsen en
Betsen
785367 785914
0785.367-0785.914-AvecBetsenEn.wav
787.384 ; spk2 ; sur Contepomi et quand on avance à l'impact
Contepomi
787384 789417
0787.384-0789.417-SurContepomiEtQuandOnAvanceÀL'impact.wav
796.327 ; spk2 ; et là Skrela qui s'engage avec
Skrela
796327 798119
0796.327-0798.119-EtLàSkrelaQuiS'engageAvec.wav
798.478 ; spk2 ; Traill

1524.774 ; spk1 ; par Jauzion
Jauzion
1524774 1525492
1524.774-1525.492-ParJauzion.wav
1544.145 ; spk1 ; et Contepomi va tenter sa chance
Contepomi
1544145 1545563
1544.145-1545.563-EtContepomiVaTenterSaChance.wav
1553.944 ; spk2 ; ah ça existe plus mon cher Thierry aujourd'hui prendre la
Thierry
1553944 1555826
1553.944-1555.826-AhÇaExistePlusMonCherThierryAujourd'huiPrendreLa.wav
1582.288 ; spk1 ; de Felipe Contepomi
Felipe Contepomi
1582288 1583720
1582.288-1583.720-DeFelipeContepomi.wav
1606.078 ; spk3 ; Thierry
Thierry
1606078 1606529
1606.078-1606.529-Thierry.wav
1606.737 ; spk3 ; Damien Traille il
Damien Traille
1606737 1607720
1606.737-1607.720-DamienTrailleIl.wav
1607.72 ; spk3 ; va faire sa rentrée à nouveau donc Fred Michalak va céder sa place
Fred Michalak
1607720 1610895
1607.720-1610.895-VaFaireSaRentréeÀNouveauDoncFredMichalakVaCéderSaPlace.wav
1613.944 ; spk1 ; de départ avec David Skrela à l'ouverture
David Skrela
1613944 1616060
1613.944-1616.060-DeDépartAvecDavidSkre

2339.78 ; spk1 ; face à Borges
Borges
2339780 2341121
2339.780-2341.121-FaceÀBorges.wav
2343.619 ; spk1 ; Christophe Dominici
Christophe Dominici
2343619 2344913
2343.619-2344.913-ChristopheDominici.wav
2350.977 ; spk1 ; et David Skrela qui a pas son jeu au pied habituel
David Skrela
2350977 2354167
2350.977-2354.167-EtDavidSkrelaQuiAPasSonJeuAuPiedHabituel.wav
2368.24 ; spk1 ; Traille et Jauzion hein
Traille
2368240 2368963
2368.240-2368.963-TrailleEtJauzionHein.wav
2379.2 ; spk1 ; hé oui dans les vingt-deux mètres français remise en jeu Ledesma
Ledesma
2379200 2382008
2379.200-2382.008-HéOuiDansLesVingt-deuxMètresFrançaisRemiseEnJeuLedesma.wav
2382.528 ; spk1 ; et Rémy Martin qui vient batailler encore
Rémy Martin
2382528 2384415
2382.528-2384.415-EtRémyMartinQuiVientBataillerEncore.wav
2382.528 ; spk2 ; et Rémy Martin oui
Rémy Martin
2382528 2384415
2382.528-2384.415-EtRémyMartinOui.wav
2384.78 ; spk1 ; et qui gagne ce ballon pour euh Ibañez
Ibañez
2384780 2386966
2384.780-2386.966-

3932.251 ; spk1 ; Pelous est au point de chute il est dépassé par le ballon
Pelous
3932251 3934676
3932.251-3934.676-PelousEstAuPointDeChuteIlEstDépasséParLeBallon.wav
3939.886 ; spk1 ; de Hernandez soient très bien tapées 
Hernandez
3939886 3941592
3939.886-3941.592-DeHernandezSoientTrèsBienTapées.wav
3944.227 ; spk1 ; allez Dominici à la relance
Dominici
3944227 3945694
3944.227-3945.694-AllezDominiciÀLaRelance.wav
3946.257 ; spk1 ; avec Heymans il a des appuis de feu
Heymans
3946257 3948015
3946.257-3948.015-AvecHeymansIlADesAppuisDeFeu.wav
3957.682 ; spk1 ; Hernandez 
Hernandez
3957682 3958808
3957.682-3958.808-Hernandez.wav
3960.689 ; spk1 ; Jauzion a prolongé au pied
Jauzion
3960689 3962150
3960.689-3962.150-JauzionAProlongéAuPied.wav
3964.755 ; spk1 ; il faut le bloquer et Thion
Thion
3964755 3965977
3964.755-3965.977-IlFautLeBloquerEtThion.wav
3972.133 ; spk1 ; Pichot
Pichot
3972133 3972706
3972.133-3972.706-Pichot.wav
3974.247 ; spk1 ; Hernandez
Hernandez
3974247 3975259
3974.

4817.001 ; spk3 ; et surtout David Skrela qui se plaint de son avant-bras
David Skrela
4817001 4819460
4817.001-4819.460-EtSurtoutDavidSkrelaQuiSePlaintDeSonAvant-bras.wav
4820.165 ; spk1 ; ouais puis peut-être même de ses jambes on on l'a à l'image euh Fabrice
Fabrice
4820165 4823902
4820.165-4823.902-OuaisPuisPeut-êtreMêmeDeSesJambesOnOnL'aÀL'imageEuhFabrice.wav
4825.505 ; spk1 ; David Skrela
David Skrela
4825505 4826344
4825.505-4826.344-DavidSkrela.wav
4830.058 ; spk1 ;  je veux parler d'Ibañez et de Pelous
Ibañez
4830058 4831554
4830.058-4831.554-JeVeuxParlerD'ibañezEtDePelous.wav
4831.94 ; spk1 ; c'est Betsen qui fait partie du comité des sages
Betsen
4831940 4834545
4831.940-4834.545-C'estBetsenQuiFaitPartieDuComitéDesSages.wav
4845.561 ; spk2 ; et Szarzewski* en fait c'est Szarzewski qui
Szarzewski
4845561 4848072
4845.561-4848.072-EtSzarzewski*EnFaitC'estSzarzewskiQui.wav
4854.64 ; spk1 ; et l'en-avant après de Betsen
Betsen
4854640 4856234
4854.640-4856.234-EtL'en-avantAprèsD

5410.973 ; spk1 ; qui va essayer de prendre de vitesse Corleto
Corleto
5410973 5413041
5410.973-5413.041-QuiVaEssayerDePrendreDeVitesseCorleto.wav
5413.499 ; spk1 ; Corleto en difficulté qui va garder le ballon au sol
Corleto
5413499 5416437
5413.499-5416.437-CorletoEnDifficultéQuiVaGarderLeBallonAuSol.wav
5428.45 ; spk2 ; ça c'est du coup de pied intelligent de Michalak il a vu il a regardé
Michalak
5428450 5431177
5428.450-5431.177-ÇaC'estDuCoupDePiedIntelligentDeMichalakIlAVuIlARegardé.wav
5442.941 ; spk1 ; alors Messieurs Landreau et Lacroix question tactique
Landreau
5442941 5445823
5442.941-5445.823-AlorsMessieursLandreauEtLacroixQuestionTactique.wav
5464.797 ; spk1 ; Fabrice Landreau d'accord
Fabrice Landreau
5464797 5465901
5464.797-5465.901-FabriceLandreauD'accord.wav
5471.441 ; spk3 ; de Contepomi mais Rodrigo Roncero qui est juste à côté de moi me dit qu'il est cuit
Contepomi
5471441 5475411
5471.441-5475.411-DeContepomiMaisRodrigoRonceroQuiEstJusteÀCôtéDeMoiMeDitQu'ilEstCui

6009.069 ; spk1 ; que cette pénalité de Felippe Contepomi
Contepomi
6009069 6011460
6009.069-6011.460-QueCettePénalitéDeFelippeContepomi.wav
6030.431 ; spk1 ; c'est parti pour Contepomi
Contepomi
6030431 6033329
6030.431-6033.329-C'estPartiPourContepomi.wav
6049.49 ; spk1 ;  Thierry
Thierry
6049490 6049935
6049.490-6049.935-Thierry.wav
6062.134 ; spk1 ; Contepomi toujours dans les airs 
Contepomi
6062134 6064080
6062.134-6064.080-ContepomiToujoursDansLesAirs.wav
6065.188 ; spk1 ; quand c'est pas lui c'est Hernandez
Hernandez
6065188 6066739
6065.188-6066.739-QuandC'estPasLuiC'estHernandez.wav
6071.871 ; spk1 ; Elissalde a récupéré
Elissalde
6071871 6073248
6071.871-6073.248-ElissaldeARécupéré.wav
6075.292 ; spk1 ; avec Heymans qui a les 
Heymans
6075292 6076185
6075.292-6076.185-AvecHeymansQuiALes.wav
6086.994 ; spk1 ; avec de Villiers
de Villiers
6086994 6088465
6086.994-6088.465-AvecDeVilliers.wav
6089.477 ; spk1 ; avec maintenant Dominici
Dominici
6089477 6091507
6089.477-6091.507-A

In [ ]:
nbToursExtraCourtsNAME, nbToursCourtsNAME, nbToursNAME